In [1]:
houses = ['Ursaia', 'Nocturna', 'Ianthe', 'Triton', 'Ankaa', 'Saren']

In [ ]:
import pandas
# Read CSV

In [ ]:
import hashlib
# Hash and Sort Names

# For each input: take name and hash, then sort

In [ ]:
# Populate adjacency list

In [ ]:
# Populate function with arguments
# Major Faculty JC

In [ ]:
# Colouring Algorithm

In [ ]:
# Output